# Julia for Data Analysis

## Bogumił Kamiński

# Lecture 3. Julia's support for scaling projects

## Understanding Julia's type system

### A single function can have multiple methods

In [1]:
methods(cd)

# 4 methods for generic function "cd":
[1] cd() in Base.Filesystem at file.jl:92
[2] cd(dir::AbstractString) in Base.Filesystem at file.jl:87
[3] cd(f::Function) in Base.Filesystem at file.jl:145
[4] cd(f::Function, dir::AbstractString) in Base.Filesystem at file.jl:95

### Every function has its own type that is a subtype of `Function`

In [2]:
sum isa Function

true

In [3]:
typeof(sum)

typeof(sum) (singleton type of function sum, subtype of Function)

In [4]:
typeof(sum) == Function

false

### Types in Julia are arranged in a hierarchy

In [5]:
supertype(typeof(sum))

Function

In [6]:
function print_supertypes(T)
    println(T)
    T == Any || print_supertypes(supertype(T))
    return nothing
end

print_supertypes (generic function with 1 method)

In [7]:
print_supertypes(Int64)

Int64
Signed
Integer
Real
Number
Any


In [8]:
function print_subtypes(T, indent_level=0)
    println(" " ^ indent_level, T)
    for S in subtypes(T)
        print_subtypes(S, indent_level + 2)
    end
    return nothing
end

print_subtypes (generic function with 2 methods)

In [9]:
print_subtypes(Integer)

Integer
  Bool
  Signed
    BigInt
    Int128
    Int16
    Int32
    Int64
    Int8
  Unsigned
    UInt128
    UInt16
    UInt32
    UInt64
    UInt8


### Union of types

In [10]:
Union{Signed, Unsigned}

Union{Signed, Unsigned}

In [11]:
Union{String, Missing}

Union{Missing, String}

### Finding a common supertype for values

In [12]:
print_supertypes(typeof([1.0, 2.0, 3.0]))

Vector{Float64}
DenseVector{Float64}
AbstractVector{Float64}
Any


In [13]:
print_supertypes(typeof(1:3))

UnitRange{Int64}
AbstractUnitRange{Int64}
OrdinalRange{Int64, Int64}
AbstractRange{Int64}
AbstractVector{Int64}
Any


In [14]:
AbstractVector

AbstractVector (alias for AbstractArray{T, 1} where T)

In [15]:
typejoin(typeof([1.0, 2.0, 3.0]), typeof(1:3))

AbstractVector (alias for AbstractArray{T, 1} where T)

## Multiple dispatch in Julia

### Defining methods of a function

In [16]:
fun(x) = println("unsupported type")

fun (generic function with 1 method)

In [17]:
fun(x::Number) = println("a number was passed")

fun (generic function with 2 methods)

In [18]:
fun(x::Float64) = println("a Float64 value")

fun (generic function with 3 methods)

In [19]:
methods(fun)

# 3 methods for generic function "fun":
[1] fun(x::Float64) in Main at In[18]:1
[2] fun(x::Number) in Main at In[17]:1
[3] fun(x) in Main at In[16]:1

In [20]:
fun("hello!")

unsupported type


In [21]:
fun(1)

a number was passed


In [22]:
fun(1.0)

a Float64 value


### Method ambiguity problem

In [23]:
bar(x, y) = "no numbers passed"

bar (generic function with 1 method)

In [24]:
bar(x::Number, y) = "first argument is a number"

bar (generic function with 2 methods)

In [25]:
bar(x, y::Number) = "second argument is a number"

bar (generic function with 3 methods)

In [26]:
bar("hello", "world")

"no numbers passed"

In [27]:
bar(1, "world")

"first argument is a number"

In [28]:
bar("hello", 2)

"second argument is a number"

In [29]:
bar(1, 2)

LoadError: MethodError: bar(::Int64, ::Int64) is ambiguous. Candidates:
  bar(x::Number, y) in Main at In[24]:1
  bar(x, y::Number) in Main at In[25]:1
Possible fix, define
  bar(::Number, ::Number)

In [30]:
 bar(x::Number, y::Number) = "both arguments are numbers"

bar (generic function with 4 methods)

In [31]:
bar(1, 2)

"both arguments are numbers"

In [32]:
methods(bar)

# 4 methods for generic function "bar":
[1] bar(x::Number, y::Number) in Main at In[30]:1
[2] bar(x::Number, y) in Main at In[24]:1
[3] bar(x, y::Number) in Main at In[25]:1
[4] bar(x, y) in Main at In[23]:1

### Improved implementation of Winsorized mean

In [33]:
function winsorized_mean(x::AbstractVector, k::Integer)
    k >= 0 || throw(ArgumentError("k must be non-negative"))
    length(x) > 2 * k || throw(ArgumentError("k is too large"))
    y = sort!(collect(x))
    for i in 1:k
        y[i] = y[k + 1]
        y[end - i + 1] = y[end - k]
    end
    return sum(y) / length(y)
end

winsorized_mean (generic function with 1 method)

In [34]:
winsorized_mean([8, 3, 1, 5, 7], 1)

5.0

In [35]:
winsorized_mean(1:10, 2)

5.5

In [36]:
winsorized_mean(1:10, "a")

LoadError: MethodError: no method matching winsorized_mean(::UnitRange{Int64}, ::String)
[0mClosest candidates are:
[0m  winsorized_mean(::AbstractVector, [91m::Integer[39m) at In[33]:1

In [37]:
winsorized_mean(10, 1)

LoadError: MethodError: no method matching winsorized_mean(::Int64, ::Int64)
[0mClosest candidates are:
[0m  winsorized_mean([91m::AbstractVector[39m, ::Integer) at In[33]:1

In [38]:
winsorized_mean(1:10, -1)

LoadError: ArgumentError: k must be non-negative

In [39]:
winsorized_mean(1:10, 5)

LoadError: ArgumentError: k is too large

## Working with packages and modules

### Including files

### Defining modules

In [40]:
module ExampleModule
    function example()
        println("Hello")
    end
end # ExampleModule

Main.ExampleModule

### Using packages

In [41]:
import Statistics
x = [1, 2, 3]

3-element Vector{Int64}:
 1
 2
 3

In [42]:
mean(x)

LoadError: UndefVarError: mean not defined

In [43]:
Statistics.mean(x)

2.0

In [44]:
using Statistics
mean(x)

2.0

### Using the STATSBASE.JL package to compute Winsorized mean

In [45]:
using Statistics
using StatsBase

In [46]:
? winsor

search: winsor winsor! winsorized_mean



```
winsor(x::AbstractVector; prop=0.0, count=0)
```

Return an iterator of all elements of `x` that replaces either `count` or proportion `prop` of the highest elements with the previous-highest element and an equal number of the lowest elements with the next-lowest element.

The number of replaced elements could be smaller than specified if several elements equal the lower or upper bound.

To compute the Winsorized mean of `x` use `mean(winsor(x))`.

# Example

```julia
julia> collect(winsor([5,2,3,4,1], prop=0.2))
5-element Array{Int64,1}:
 4
 2
 3
 4
 2
```


In [47]:
mean(winsor([8, 3, 1, 5, 7], count=1))

5.0

## Using macros

### Macro syntax

In [48]:
@time 1 + 2

  0.000000 seconds


3

In [49]:
@time(1 + 2)

  0.000001 seconds


3

In [50]:
@assert 1 == 2 "1 is not equal 2"

LoadError: AssertionError: 1 is not equal 2

In [51]:
@assert(1 == 2, "1 is not equal 2")

LoadError: AssertionError: 1 is not equal 2

In [52]:
@macroexpand @assert(1 == 2, "1 is not equal 2")

:(if 1 == 2
      nothing
  else
      Base.throw(Base.AssertionError("1 is not equal 2"))
  end)

In [53]:
@macroexpand @time 1 + 2

quote
    #= timing.jl:216 =#
    while false
        #= timing.jl:216 =#
    end
    #= timing.jl:217 =#
    local var"#39#stats" = Base.gc_num()
    #= timing.jl:218 =#
    local var"#41#elapsedtime" = Base.time_ns()
    #= timing.jl:219 =#
    local var"#42#compile_elapsedtime" = Base.cumulative_compile_time_ns_before()
    #= timing.jl:220 =#
    local var"#40#val" = $(Expr(:tryfinally, :(1 + 2), quote
    var"#41#elapsedtime" = Base.time_ns() - var"#41#elapsedtime"
    #= timing.jl:222 =#
    var"#42#compile_elapsedtime" = Base.cumulative_compile_time_ns_after() - var"#42#compile_elapsedtime"
end))
    #= timing.jl:224 =#
    local var"#43#diff" = Base.GC_Diff(Base.gc_num(), var"#39#stats")
    #= timing.jl:225 =#
    Base.time_print(var"#41#elapsedtime", (var"#43#diff").allocd, (var"#43#diff").total_time, Base.gc_alloc_count(var"#43#diff"), var"#42#compile_elapsedtime", true)
    #= timing.jl:226 =#
    var"#40#val"
end

### Benchmarking with macros

In [54]:
using BenchmarkTools
x = rand(10^6); # use ; at the end of the expression passed in REPL to suppress printing its value to the terminal
@benchmark winsorized_mean($x, 10^5) # since x is a global variable use $x to ensure proper benchmarking of the tested code

BenchmarkTools.Trial: 90 samples with 1 evaluation.
 Range (min … max):  53.340 ms … 58.776 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     55.912 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   55.910 ms ±  1.484 ms  ┊ GC (mean ± σ):  0.59% ± 1.30%

    █ ▁  ▄▁▁   ▄      ▄█ █ ▁▄  ▄▄ ▁     ▁▄▁▁▁▁  ▁█      ▁▁▁    
  ▆▆█▆█▁▆███▆▁▆█▆▁▆▆▆▆██▆█▁██▁▆██▁█▆▆▆▆▆██████▆▆██▁▆▆▆▁▁███▁▆ ▁
  53.3 ms         Histogram: frequency by time        58.6 ms <

 Memory estimate: 7.63 MiB, allocs estimate: 2.

In [55]:
using Statistics
using StatsBase
@benchmark mean(winsor($x; count=10^5))

BenchmarkTools.Trial: 347 samples with 1 evaluation.
 Range (min … max):  13.046 ms … 23.587 ms  ┊ GC (min … max): 0.00% … 15.44%
 Time  (median):     13.988 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   14.426 ms ±  1.228 ms  ┊ GC (mean ± σ):  2.43% ±  4.88%

       ██▂▆                                                    
  ▄▅▃▄▄████▇▇▅▇▆▇▅▄▃▃▃▄▃▃▃▄▄▅▃▄▃▂▃▃▃▃▃▂▂▂▂▁▂▂▁▁▁▁▁▁▁▂▁▁▁▁▂▁▁▂ ▃
  13 ms           Histogram: frequency by time          19 ms <

 Memory estimate: 7.63 MiB, allocs estimate: 2.

In [56]:
@btime mean(winsor($x; count=10^5))

  12.982 ms (2 allocations: 7.63 MiB)


0.4997748954801521